In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 🧩 AGI Voice Agent - Full End-to-End Pipeline (Notebook 19)

### Objective:
Integrate all components:
- Memory retrieval
- RAG (Retrieval-Augmented Generation)
- Reasoning via LLM
- Voice input (transcription)
- Text-to-voice response


---

✅ 1. Install Required Libraries

pip install transformers sentence-transformers langchain whisper elevenlabs streamlit fastapi uvicorn


---

✅ 2. Initialize Components

from sentence_transformers import SentenceTransformer
from transformers import pipeline
import whisper
from elevenlabs import generate, set_api_key

# Models
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
llm = pipeline('text-generation', model='gpt2')
whisper_model = whisper.load_model("base")
set_api_key("YOUR_ELEVENLABS_API_KEY")




✅ 3. Voice to Text

def transcribe_audio(audio_path):
    result = whisper_model.transcribe(audio_path)
    return result['text']




✅ 4. Retrieve from Memory / Vector DB (Example with FAISS)

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

retriever = FAISS.load_local("vectorstore/agi_voice_agent_faiss", HuggingFaceEmbeddings("all-MiniLM-L6-v2"))

def retrieve_context(query, k=3):
    results = retriever.similarity_search(query, k=k)
    return "\n".join([r.page_content for r in results])




✅ 5. Reasoning Step

def reason_with_context(query):
    context = retrieve_context(query)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer step by step:"
    response = llm(prompt)
    return response[0]['generated_text']




✅ 6. Text to Voice

def text_to_speech(text, output_file="outputs/response.mp3"):
    audio = generate(text=text, voice="Rachel")
    with open(output_file, 'wb') as f:
        f.write(audio)
    return output_file




✅ 7. Full Pipeline Function

def agi_voice_agent(audio_input_path=None, text_input=None):
    if audio_input_path:
        text = transcribe_audio(audio_input_path)
    else:
        text = text_input

    reasoning_output = reason_with_context(text)
    voice_output = text_to_speech(reasoning_output)
    return reasoning_output, voice_output




✅ 8. Example Run

reasoning, voice_path = agi_voice_agent(text_input="What is the theory of consciousness?")
print("Reasoning:", reasoning)
print("Voice output saved at:", voice_path )


SyntaxError: invalid character '✅' (U+2705) (929202878.py, line 14)